In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\backe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
kalimat = [
    "I love Indonesia.",
    "Sate is scrumptious.",
    "Sei Caduto in un Errore Logico.",
    "Sugeng rawuh! sedulur sak kabehe.",
    "Punten aa, kumaha ieu teh?"
]

# Tokenization⛓️‍💥
**How to mecah texts to unit terkecilnya bisa kata, subkata or character.**

## Tradisional Approach

Pendekatan pertama yg bisa dipake itu adalah as simple kita pecah2 dia berdasarkan spasi, tapi ingat ga smua kalimat yg kita pisah utk jadi kata in general bisa kita pisah spt itu

In [9]:
for kata in kalimat:
    kata_kata = kata.split()
    print(f'Kalimat: {kata}')
    print(f'Kata: {kata_kata}')

Kalimat: I love Indonesia.
Kata: ['I', 'love', 'Indonesia.']
Kalimat: Sate is scrumptious.
Kata: ['Sate', 'is', 'scrumptious.']
Kalimat: Sei Caduto in un Errore Logico.
Kata: ['Sei', 'Caduto', 'in', 'un', 'Errore', 'Logico.']
Kalimat: Sugeng rawuh! sedulur sak kabehe.
Kata: ['Sugeng', 'rawuh!', 'sedulur', 'sak', 'kabehe.']
Kalimat: Punten aa, kumaha ieu teh?
Kata: ['Punten', 'aa,', 'kumaha', 'ieu', 'teh?']


Nah kek gitu kalo dia pakenya split, simple sebenernya disini konsep dia itu adlah misahinnya based delimiternya(default->spasi). Tapi bisa kita liat disini minesnya dia bner2 gabisa handle tanda baca dg bagus, liat si titik selalu ngikut di kata terakhir and etc. Makanya ada teknik yg lebih bagus yaitu pake Word Tokenizer

In [10]:
for kata in kalimat:
    kata_kata = word_tokenize(kata)
    print(f'Kalimat: {kata}')
    print(f'Kata: {kata_kata}')

Kalimat: I love Indonesia.
Kata: ['I', 'love', 'Indonesia', '.']
Kalimat: Sate is scrumptious.
Kata: ['Sate', 'is', 'scrumptious', '.']
Kalimat: Sei Caduto in un Errore Logico.
Kata: ['Sei', 'Caduto', 'in', 'un', 'Errore', 'Logico', '.']
Kalimat: Sugeng rawuh! sedulur sak kabehe.
Kata: ['Sugeng', 'rawuh', '!', 'sedulur', 'sak', 'kabehe', '.']
Kalimat: Punten aa, kumaha ieu teh?
Kata: ['Punten', 'aa', ',', 'kumaha', 'ieu', 'teh', '?']


Keliatan kan disini bedanya, si word tokenizer itu udah misah juga sekalian sama tanda bacanya. Tapi disini problem karena secara semantik suatu kata itu ga hanya dipisah dengan "suatu tanda baca" atau unit dasarnya. Misal nih kek apple's itu nanti dia jadinya apple,',s

Satu lagi haha, si word tokenizer ini juga "hancur" ketika kita punya bahasa baru/gaul gt2lah. Misal -> `Lu kerja dimans sii? kok keliatanya gabut btz.` Nah di kamus word tokenizer itu kata2 spt dimans, gabut, btz gaada jadi dia bisa anggap sbg OOV dan nanti dia jadiin itu UNK token.

## BPE in Action 

Nah model modern sekarang itu mostly pakenya Byte Pair Encoding, jadi basisnya itu udah subword ngetokenizenya. Misal Mahasiswa -> 'Maha', '##siswa'

In [11]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\backe\miniconda3\envs\dl_iwan\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\backe\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
encoded = tokenizer(kalimat, padding=True, 
                    truncation=True, return_tensors="pt")

for i, teks in enumerate(kalimat):
    # ambil input ids utk kalimat ke-i
    input_ids_kalimat = encoded['input_ids'][i].tolist()
    # convert id token jadi string token lagi
    token_kalimat = tokenizer.convert_ids_to_tokens(input_ids_kalimat)
    print(f'Kalimat:{teks}')
    print(f'Kata: {token_kalimat}')

Kalimat:I love Indonesia.
Kata: ['[CLS]', 'i', 'love', 'indonesia', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Kalimat:Sate is scrumptious.
Kata: ['[CLS]', 'sat', '##e', 'is', 'sc', '##rum', '##pt', '##ious', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Kalimat:Sei Caduto in un Errore Logico.
Kata: ['[CLS]', 'se', '##i', 'cad', '##uto', 'in', 'un', 'error', '##e', 'logic', '##o', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Kalimat:Sugeng rawuh! sedulur sak kabehe.
Kata: ['[CLS]', 'su', '##gen', '##g', 'raw', '##uh', '!', 'se', '##du', '##lu', '##r', 'sa', '##k', 'ka', '##be', '##he', '.', '[SEP]']
Kalimat:Punten aa, kumaha ieu teh?
Kata: ['[CLS]', 'punt', '##en', 'aa', ',', 'ku', '##mah', '##a', 'ie', '##u', 'te', '##h', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


Oke, mantap yaa disini keliatan kalo dia treatmentnya itu dipecah2 subwordnya. Sedikit info utk token
- [CLS] -> sbg representasi klo ini tu kalimat
- [PAD] -> sbg 'pengisi' biar urutan yg pendek itu punya pjg yg sama dg urutan terpanjang dlm suatu batch
- [SEP] -> sbg penanda atau akhir dri suatu separator